In [1]:
import nbimporter
import CNNChar as CNNC
import tensorflow as tf
import numpy as np
import json

def load_yelp(alphabet):
    examples = []
    labels = []
    reviews = []
    with open('yelp_reviews_small.json') as f:
        i = 0
        for line in f:
            review = json.loads(line)
            stars = review["stars"]
            text = review["text"]
            if stars != 3:
                text_end_extracted = extract_end(list(text.lower()))
                padded = pad_sentence(text_end_extracted)
                #if i == 0: print(padded)
                reviews.append(padded)
                text_int8_repr = string_to_int8_conversion(padded, alphabet)
                if stars == 1 or stars == 2:
                    labels.append([1, 0])
                    examples.append(text_int8_repr)
                elif stars == 4 or stars == 5:
                    labels.append([0, 1])
                    examples.append(text_int8_repr)
                i += 1
                if i % 10000 == 0:
                    print("Non-neutral instances processed: " + str(i))
        print("Non-neutral instances processed: " + str(i))
    return examples, labels, reviews

def extract_end(char_seq):
    if len(char_seq) > 1014:
        char_seq = char_seq[-1014:]
    return char_seq

def pad_sentence(char_seq, padding_char=" "):
    char_seq_length = 1014
    num_padding = char_seq_length - len(char_seq)
    new_char_seq = char_seq + [padding_char] * num_padding
    return new_char_seq

def string_to_int8_conversion(char_seq, alphabet):
    x = np.array([alphabet.find(char) for char in char_seq], dtype=np.int8)
    return x

def get_batched_one_hot(char_seqs_indices, labels, start_index, end_index):
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}\n"
    x_batch = char_seqs_indices[start_index:end_index]
    y_batch = labels[start_index:end_index]
    x_batch_one_hot = np.zeros(shape=[len(x_batch), len(alphabet), len(x_batch[0]), 1])
    for example_i, char_seq_indices in enumerate(x_batch):
        for char_pos_in_seq, char_seq_char_ind in enumerate(char_seq_indices):
            if char_seq_char_ind != -1:
                x_batch_one_hot[example_i][char_seq_char_ind][char_pos_in_seq][0] = 1
    return [x_batch_one_hot, y_batch]

def load_data():
    # TODO Add the new line character later for the yelp'cause it's a multi-line review
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}\n"
    examples, labels, reviews = load_yelp(alphabet)
    x = np.array(examples, dtype=np.int8) # Convert list into np array
    y = np.array(labels, dtype=np.int8) # Convert list into np array
    print("x_char_seq_ind=" + str(x.shape))
    print("y shape=" + str(y.shape))
    return [x, y, reviews]

def batch_iter(x, y, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    # data = np.array(data)
    data_size = len(x)
    num_batches_per_epoch = int(data_size/batch_size) + 1
    for epoch in range(num_epochs):
        print("In epoch >> " + str(epoch + 1))
        print("num batches per epoch is: " + str(num_batches_per_epoch))
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            x_shuffled = x[shuffle_indices]
            y_shuffled = y[shuffle_indices]
        else:
            x_shuffled = x
            y_shuffled = y
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            x_batch, y_batch = get_batched_one_hot(x_shuffled, y_shuffled, start_index, end_index)
            batch = [x_batch, y_batch]
            yield batch


batch_size = 100
num_epochs = 2
[x, y, reviews] = load_data()
print("x=%s,\n y=%s" % (np.array_str(x), np.array_str(y)))
bg = batch_iter(x, y, batch_size, num_epochs, shuffle=False)


Importing Jupyter notebook from CNNChar.ipynb
Non-neutral instances processed: 812
x_char_seq_ind=(812, 1014)
y shape=(812, 2)
x=[[ 3 17 39 ..., -1 -1 -1]
 [20 13  5 ..., -1 -1 -1]
 [ 3 17 39 ..., -1 -1 -1]
 ..., 
 [22 -1 18 ..., 17  4 39]
 [19  7  8 ..., -1 -1 -1]
 [19  7  4 ..., -1 -1 -1]],
 y=[[0 1]
 [1 0]
 [0 1]
 ..., 
 [0 1]
 [0 1]
 [1 0]]


In [3]:
# Batch 1 
# ''.join(reviews[0])
# x[0][0:100]
# [v[0][0] for v in x_batch[0]]
# w1 = sess.run("conv-maxpool-1/W:0", feed_dict={cnn.input_x:x_batch})
# c1 = sess.run("conv-maxpool-1/conv1:0", feed_dict={cnn.input_x:x_batch})
# for i, j in enumerate(x[0][0:7]): print i, j, x_batch[0][j][i][0], w1[j][i][0][0]
# s=0; for i, j in enumerate(x[0][0:7]): s+= x_batch[0][j][i][0]*w1[j][i][0][0]

[x_batch, y_batch] = next(bg)

print(x_batch[0])
print(x_batch.shape)
# CNN
cnn = CNNC.CharCNN()
sess = tf.Session();
sess.run(tf.global_variables_initializer())

writer = tf.summary.FileWriter("./char_cnn", graph=tf.get_default_graph())
sess.run("conv-maxpool-1/pool1:0", feed_dict={cnn.input_x:x_batch})

w1 = sess.run("conv-maxpool-1/W:0", feed_dict={cnn.input_x:x_batch})
b1 = sess.run("conv-maxpool-1/b:0", feed_dict={cnn.input_x:x_batch})
w2 = sess.run("conv-maxpool-2/W:0", feed_dict={cnn.input_x:x_batch})
b2 = sess.run("conv-maxpool-2/b:0", feed_dict={cnn.input_x:x_batch})
w3 = sess.run("conv-3/W:0", feed_dict={cnn.input_x:x_batch})
b3 = sess.run("conv-3/b:0", feed_dict={cnn.input_x:x_batch})
w4 = sess.run("conv-4/W:0", feed_dict={cnn.input_x:x_batch})
b4 = sess.run("conv-4/b:0", feed_dict={cnn.input_x:x_batch})
w5 = sess.run("conv-5/W:0", feed_dict={cnn.input_x:x_batch})
b5 = sess.run("conv-5/b:0", feed_dict={cnn.input_x:x_batch})
w6 = sess.run("conv-maxpool-6/W:0", feed_dict={cnn.input_x:x_batch})
b6 = sess.run("conv-maxpool-6/b:0", feed_dict={cnn.input_x:x_batch})
w7 = sess.run("fc-1/W:0", feed_dict={cnn.input_x:x_batch})
b7 = sess.run("fc-1/b:0", feed_dict={cnn.input_x:x_batch})
w8 = sess.run("fc-2/W:0", feed_dict={cnn.input_x:x_batch})
b8 = sess.run("fc-2/b:0", feed_dict={cnn.input_x:x_batch})
w9 = sess.run("fc-3/W:0", feed_dict={cnn.input_x:x_batch})
b9 = sess.run("fc-3/b:0", feed_dict={cnn.input_x:x_batch})
c1 = sess.run("conv-maxpool-1/conv1:0", feed_dict={cnn.input_x:x_batch})

[[[ 0.]
  [ 0.]
  [ 0.]
  ..., 
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ..., 
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ..., 
  [ 0.]
  [ 0.]
  [ 0.]]

 ..., 
 [[ 0.]
  [ 0.]
  [ 0.]
  ..., 
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ..., 
  [ 0.]
  [ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ..., 
  [ 0.]
  [ 0.]
  [ 0.]]]
(100, 70, 1014, 1)


InvalidArgumentError: You must feed a value for placeholder tensor 'input_x' with dtype float
	 [[Node: input_x = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'input_x', defined at:
  File "/Users/fperez/.pyenv/versions/anaconda2-4.3.0/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/fperez/.pyenv/versions/anaconda2-4.3.0/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/fperez/.pyenv/versions/anaconda2-4.3.0/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/fperez/.pyenv/versions/anaconda2-4.3.0/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/fperez/.pyenv/versions/anaconda2-4.3.0/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().st

In [ ]:
print(x_batch.shape)
print(x_batch[0][0])

In [ ]:
# First 100 chars of example 1 coded to positions in the alphabet:
# alphabet: "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}\n"
# example 1: "dr. goldberg offers everything i look for in a general..."
x[0][0:100]

In [ ]:
[v[0][0] for v in x_batch[0]] # Character representation for 'd' in "dr."
#[v[1][0] for v in x_batch[0]] # Character representation for 'r' in "dr."
#[v[2][0] for v in x_batch[0]] # Character representation for '.' in "dr."

In [ ]:
# 70 (Filter height or alphabet lenght) x 7 (filter width or chars per filter) x 1 (depth) x 256 filters
w1 = sess.run("conv-maxpool-1/W:0", feed_dict={cnn.input_x:x_batch})
w1.shape

In [ ]:
# Result of applying the convolution
# 512 samples x (1 dim vector x 1008 values after applying the filter) x 256 filters
c1 = sess.run("conv-maxpool-1/conv1:0", feed_dict={cnn.input_x:x_batch})
c1.shape

In [ ]:
# How to check if we're doin fine?
# E.g. take the first value fo the first sample (of 512) on the first filter (of 256)
c1[0][0][0][0]
#  s  x  y  f

In [ ]:
# Then perform the operation "manually" of the first filter (7 chars) to see what's in there required for the calculation:
for i, j in enumerate(x[0][0:7]): print i, j, x_batch[0][j][i][0], w1[j][i][0][0]
# Note that the 3rd element multiplies by 0 so it won't be taken into account
# Remember that x[0] is the first example, so x[0][0:7] are the first 7 chars of the first example: "dr. gol"

In [ ]:
# Then perform the manual calculation and observe that the result is the same
s=0
for i, j in enumerate(x[0][0:7]): s+= x_batch[0][j][i][0]*w1[j][i][0][0]
print(s)